# Deepchem MPNN

## Featurizing molecules

In [2]:
!pip install --pre deepchem
!pip install wandb --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import deepchem as dc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# RDKit
import rdkit
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem.Draw import SimilarityMaps

import os
from pathlib import Path
import csv
import math

In [4]:
content_path = Path('/content/drive/MyDrive/chem_data')
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# !pip install -qU deepchem wandb

In [6]:
import wandb
wandb.login()

import warnings
warnings.filterwarnings('ignore')

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: venustheplanet. Use `wandb login --relogin` to force relogin


In [7]:
from deepchem.models.wandblogger import WandbLogger

In [8]:
# Create directories
# def create_dir():
""" Create directories featurized, valid, train, test.."""


# create_dir()
mpnn_dir = content_path/'mpnn_dir'
if not mpnn_dir.is_dir():
  mpnn_dir.mkdir()

basic_gcn_dir = content_path/'basic_gcn_dir'
if not basic_gcn_dir.is_dir():
  basic_gcn_dir.mkdir()

In [9]:
# Featurize data into dc dataset
def csv_to_feat(filename, feat="mpnn"):

  data = pd.read_csv(f"{content_path}/{filename}.csv")
  smiles = data['smiles'].tolist()
  pki = data['pki'].tolist()
  if feat == "mpnn":
    featurizer = dc.feat.MolGraphConvFeaturizer(use_edges=True)
  elif feat == "gcn":
    featurizer = dc.feat.ConvMolFeaturizer()
  X = featurizer.featurize(smiles)
  dataset = dc.data.NumpyDataset(X=X, y=pki)
  # print(dataset)
  # print(pki)
  return dataset

In [10]:
# Metrics

metric = dc.metrics.Metric(
    dc.metrics.mean_absolute_error, task_averager=np.mean, mode="regression")

pearson_r2_score = dc.metrics.Metric(
    dc.metrics.pearson_r2_score, task_averager=np.mean, mode="regression")

mean_absolute_error = dc.metrics.Metric(
    dc.metrics.mean_absolute_error, task_averager=np.mean, mode="regression")

rms_score = dc.metrics.Metric(
    dc.metrics.rms_score, task_averager=np.mean, mode="regression")

### Trial GCNModel

In [11]:
!pip install dgllife
# !pip install dgl
!pip install dgl-cu101 dglgo -f https://data.dgl.ai/wheels/repo.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://data.dgl.ai/wheels/repo.html


In [12]:
from deepchem.models.torch_models import MPNNModel
from deepchem.models.wandblogger import WandbLogger
from deepchem.models import GCNModel

In [13]:
def train_GCN(train, val, tid, num_layers=2, num_epoch=5):


  callback = dc.models.ValidationCallback(val, 1000, metric)
  logger = WandbLogger(project='gcn', name=tid)
  model = GCNModel(mode='regression', n_tasks=1, 
                    learning_rate=0.001, batch_size=16, graph_conv_layers=[2048]*num_layers, wandb_logger=logger)
  # model = GCNModel(mode='regression', n_tasks=1, batch_size=16, learning_rate=0.01, graph_conv_layers=[64]*num_layers)
  loss = model.fit(train, nb_epoch=num_epoch, callbacks=callback)
  logger.finish()


  model_mae = model.evaluate(val, [mean_absolute_error], per_task_metrics=False)
  model_r2 = model.evaluate(val, [pearson_r2_score], per_task_metrics=False)
  model_rmse = model.evaluate(val, [rms_score], per_task_metrics=False)

  print(f"mae: {model_mae}")
  print(f"r2: {model_r2}")
  print(f"rmse: {model_rmse}")

  return model
  # return model_mae['mean-mean_absolute_error'], model_r2['mean-pearson_r2_score'], model_rmse['mean-rms_score']

In [14]:
# trial model 



def train_MPNN(train, val, tid, num_message=3, num_epoch=5):


  callback = dc.models.ValidationCallback(val, 500, metric)
  logger = WandbLogger(project='mpnn', name=tid)
  model = MPNNModel(mode='regression', n_tasks=1, 
                    learning_rate=0.001, num_step_message_passing=num_message, wandb_logger=logger)
  # model = GCNModel(mode='regression', n_tasks=1, batch_size=16, learning_rate=0.01, graph_conv_layers=[64]*num_layers)
  loss = model.fit(train, nb_epoch=num_epoch, callbacks=callback)
  logger.finish()


  model_mae = model.evaluate(val, [mean_absolute_error], per_task_metrics=False)
  model_r2 = model.evaluate(val, [pearson_r2_score], per_task_metrics=False)
  model_rmse = model.evaluate(val, [rms_score], per_task_metrics=False)

  print(f"mae: {model_mae}")
  print(f"r2: {model_r2}")
  print(f"rmse: {model_rmse}")

  return model

In [15]:

from deepchem.models.torch_models import GATModel
def train_GAT(train, val, tid, num_epoch=5):


  callback = dc.models.ValidationCallback(val, 500, metric)
  logger = WandbLogger(project='gat', name=tid)
  model = GATModel(mode='regression', n_tasks=1, 
                    learning_rate=0.001, wandb_logger=logger)
  # model = GCNModel(mode='regression', n_tasks=1, batch_size=16, learning_rate=0.01, graph_conv_layers=[64]*num_layers)
  loss = model.fit(train, nb_epoch=num_epoch, callbacks=callback)
  logger.finish()


  model_mae = model.evaluate(val, [mean_absolute_error], per_task_metrics=False)
  model_r2 = model.evaluate(val, [pearson_r2_score], per_task_metrics=False)
  model_rmse = model.evaluate(val, [rms_score], per_task_metrics=False)

  print(f"mae: {model_mae}")
  print(f"r2: {model_r2}")
  print(f"rmse: {model_rmse}")

  return model_mae['mean-mean_absolute_error'], model_r2['mean-pearson_r2_score'], model_rmse['mean-rms_score']


In [16]:
# split dataset
def split_data(dataset):
  from deepchem.splits.splitters import RandomSplitter

  splitter = RandomSplitter()
  train, val, test = splitter.train_valid_test_split(
      dataset=dataset,
      log_every_n=100
  )
  return train, val, test


In [17]:
dataset_ids = ['11', '15', '51', '72', '87', '100', '107', '108', '114', '121', '129', '130', '136', '137', '138', '155', '165', '176', '194', '252', '259', '278', '280', '10142', '10193', '10280', '10627', '11290', '12209', '12952', '19905']
len(dataset_ids)

31

In [18]:
def basic_gcn(data):
  tid = 'tid' + data
  dataset = csv_to_feat(tid)
  train, val, test = split_data(dataset)
  model = train_GCN(train, val, tid, num_epoch=50)

  model_mae = model.evaluate(test, [mean_absolute_error], per_task_metrics=False)
  model_r2 = model.evaluate(test, [pearson_r2_score], per_task_metrics=False)
  model_rmse = model.evaluate(test, [rms_score], per_task_metrics=False)

  print('model_mae', model_mae)
  print('r2', model_r2)
  print('rmse', model_rmse)

  return model_mae, model_r2, model_rmse

In [19]:
# mae15, r215, rmse15 = basic_gcn('15')

In [20]:
# print(mae15, r215, rmse15)

In [21]:
def basic_mpnn(data):
  tid = 'tid' + data
  dataset = csv_to_feat(tid)
  train, val, test = split_data(dataset)
  train_MPNN(train, val, tid, num_epoch=50)

In [22]:
def basic_GAT(data):
  tid = 'tid' + data
  dataset = csv_to_feat(tid)
  train, val, test = split_data(dataset)
  train_GAT(train, val, tid, num_epoch=200)

In [23]:
# basic_GAT('15')

In [24]:
# basic_mpnn('15')

In [25]:
def run_mpnn(num_runs, datasets, num_epochs=50):
  # results = {}
  mean_results = {}

  for id in datasets:
    results = {}

    mae_list = []
    r2_list = []
    rmse_list = []

    tid = 'tid'+id
    dataset = csv_to_feat(tid)
    # train, val, test = split_data(dataset)
    print('starting', tid)
    
    for run in range(num_runs):
      train, val, test = split_data(dataset)
      model = train_MPNN(train, val, tid, num_epoch=num_epochs)
      mae = model.evaluate(test, [mean_absolute_error], per_task_metrics=False)
      r2 = model.evaluate(test, [pearson_r2_score], per_task_metrics=False)
      rmse = model.evaluate(test, [rms_score], per_task_metrics=False)

      print(f'mae for {run} for {tid}: {mae}')
      print(f'r2 for {run} for {tid}: {r2}')
      print(f'rmse for {run} for {tid}: {rmse}')
      
      mae_list.append(mae['mean-mean_absolute_error'])
      r2_list.append(r2['mean-pearson_r2_score'])
      rmse_list.append(rmse['mean-rms_score'])
    
    results[f'{tid}_mae'] = mae_list
    results[f'{tid}_r2'] = r2_list
    results[f'{tid}_rmse'] = rmse_list

    mean_mae = np.mean(mae_list)
    mean_r2 = np.mean(r2_list)
    mean_rmse = np.mean(rmse_list)

    mean_results[f'{tid}_mean_mae'] = mean_mae
    mean_results[f'{tid}_mean_r2'] = mean_r2
    mean_results[f'{tid}_mean_rmse'] = mean_rmse


    print(f'mean mae for {num_runs} for {tid}: {mean_mae}')
    print(f'mean r2 for {num_runs} for {tid}: {mean_r2}')
    print(f'mean rmse for {num_runs} for {tid}: {mean_rmse}')
   
    results_df = pd.DataFrame(results)
    results_df.to_csv(f"{mpnn_dir}/{tid}_mpnn.csv")

  return results, mean_results

# model_mae['mean-mean_absolute_error'], model_r2['mean-pearson_r2_score'], model_rmse['mean-rms_score']

In [26]:
basic_gcn_results, mean_basic_gcn_results = run_mpnn(3, dataset_ids)

starting tid11


train/loss,█▆▁
train/loss,0.42267


mae: {'mean-mean_absolute_error': 1.0248644318025566}
r2: {'mean-pearson_r2_score': 0.11720535541086576}
rmse: {'mean-rms_score': 1.2232705096770544}
mae for 0 for tid11: {'mean-mean_absolute_error': 1.1250902291707356}
r2 for 0 for tid11: {'mean-pearson_r2_score': 0.2327460775978384}
rmse for 0 for tid11: {'mean-rms_score': 1.3517284112199741}


train/loss,▇█▁
train/loss,0.20086


mae: {'mean-mean_absolute_error': 0.7822521668496382}
r2: {'mean-pearson_r2_score': 0.32914124123923105}
rmse: {'mean-rms_score': 0.9862190472215732}
mae for 1 for tid11: {'mean-mean_absolute_error': 0.9319351112484303}
r2 for 1 for tid11: {'mean-pearson_r2_score': 0.17200314266963498}
rmse for 1 for tid11: {'mean-rms_score': 1.1168145744059386}


...

mae: {'mean-mean_absolute_error': 0.7632330689759087}
r2: {'mean-pearson_r2_score': 0.15313190833044468}
rmse: {'mean-rms_score': 0.9376128302156634}
mae for 0 for tid19905: {'mean-mean_absolute_error': 0.7645173416749869}
r2 for 0 for tid19905: {'mean-pearson_r2_score': 3.2936281376047256e-05}
rmse for 0 for tid19905: {'mean-rms_score': 0.9630147222686775}


train/loss,██▁
train/loss,0.37358


mae: {'mean-mean_absolute_error': 0.7675914930023349}
r2: {'mean-pearson_r2_score': 0.00736055732662348}
rmse: {'mean-rms_score': 0.9431660174593637}
mae for 1 for tid19905: {'mean-mean_absolute_error': 0.7875037529150019}
r2 for 1 for tid19905: {'mean-pearson_r2_score': 0.06443700261007368}
rmse for 1 for tid19905: {'mean-rms_score': 0.9680940016812444}


train/loss,▇█▁
train/loss,0.40828


mae: {'mean-mean_absolute_error': 0.6591422164480217}
r2: {'mean-pearson_r2_score': 0.07304515963396942}
rmse: {'mean-rms_score': 0.7909375638940602}
mae for 2 for tid19905: {'mean-mean_absolute_error': 0.8013769190382302}
r2 for 2 for tid19905: {'mean-pearson_r2_score': 0.023794045544442397}
rmse for 2 for tid19905: {'mean-rms_score': 0.9733603074017187}
mean mae for 3 for tid19905: 0.7844660045427396
mean r2 for 3 for tid19905: 0.02942132814529737
mean rmse for 3 for tid19905: 0.9681563437838802


### Hyperparameter

In [27]:
!pip install hyperopt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [28]:
from hyperopt import hp, fmin, tpe, Trials


In [29]:
# search_space = {
# 'node_out_feats': ('int', [16, 2048]),
# 'edge_hidden_feats': ('int', [16, 2048]),
# 'num_step_message_passing': ('int', [1,10]),
# 'num_step_set2set': ('int', [3,24]),
# 'num_layer_set2set': ('int', [3, 24]),
# 'learning_rate': ('cont', [0.0001, 0.1]),
# # 'nb_epoch': ('int', [20, 200])
# }

search_space = {
'node_out_feats': hp.uniform('node_out_feats', low=16, high=2048),
'edge_hidden_feats': hp.uniform('edge_hidden_feats', low=16, high=2048),
'num_step_message_passing': hp.uniform('num_step_message_passing', low=1, high=10),
'num_step_set2set': hp.uniform('num_step_set2set', low=3, high=24),
'num_layer_set2set': hp.uniform('num_layer_set2set', low=3, high=24),
'learning_rate': hp.uniform('learning_rate', low=0.0001, high=0.1),
# 'nb_epoch': ('int', [20, 200])
}

In [30]:
import tempfile
#tempfile is used to save the best checkpoint later in the program.

metric_opt = dc.metrics.Metric(dc.metrics.mean_absolute_error)

def fm(args):
  save_dir = tempfile.mkdtemp()
  model = MPNNModel(n_tasks=1,
                    mode='regression',
                    node_out_feats=args['node_out_feats'],
                    edge_hidden_feats=args['edge_hidden_feats'],
                    num_step_message_passing=args['num_step_message_passing'],
                    num_step_set2set=args['num_step_set2set'],
                    num_layer_set2set=args['num_layer_set2set'],
                    learning_rate=args['learning_rate'],
                    )
  #validation callback that saves the best checkpoint, i.e the one with the maximum score.
  validation=dc.models.ValidationCallback(val, 1000, [metric],save_dir=save_dir,save_on_minimum=False)
  
  model.fit(train, nb_epoch=50,callbacks=validation)

  #restoring the best checkpoint and passing the negative of its validation score to be minimized.
  model.restore(model_dir=save_dir)
  valid_score = model.evaluate(val, [mean_absolute_error], per_task_metrics=False)

  return valid_score['mean-mean_absolute_error']

In [31]:
tid = 'tid'+'12952'
dataset = csv_to_feat(tid)
train, val, test = split_data(dataset)

In [32]:
trials=Trials()
best = fmin(fm,
    		space= search_space,
    		algo=tpe.suggest,
    		max_evals=15,
    		trials = trials)

  0%|          | 0/15 [00:00<?, ?it/s, best loss: ?]


TypeError: ignored

In [ ]:
# Hyperparameters tuning
!pip install pyGPGO
import pyGPGO
from pyGPGO.covfunc import matern32
from pyGPGO.acquisition import Acquisition
from pyGPGO.surrogates.GaussianProcess import GaussianProcess
from pyGPGO.GPGO import GPGO



In [ ]:
params_dict = {
'node_out_feats': ('int', [16, 2048]),
'edge_hidden_feats': ('int', [16, 2048]),
'num_step_message_passing': ('int', [1,10]),
'num_step_set2set': ('int', [3,24]),
'num_layer_set2set': ('int', [3, 24]),
'learning_rate': ('cont', [0.0001, 0.1]),
# 'nb_epoch': ('int', [20, 200])
}

In [ ]:
df_param = pd.DataFrame.from_dict(params_dict,orient='index', )
display(df_param)

In [ ]:
def hyper_model_mpnn(node_out_feats, edge_hidden_feats, num_step_message_passing, num_step_set2set, num_layer_set2set,
                     learning_rate, **params):
     
    global neg_mae
    global mae
    global r2
    global rmse
    global config
    
    # dense_layer_size = int(round(dense_layer_size))
    node_out_feats = int(round(node_out_feats))
    # graph_conv_depth = 2 # two graph convolutional layers
    edge_hidden_feats = int(round(edge_hidden_feats))
    num_step_message_passing = int(round(num_step_message_passing))
    num_step_set2set = int(round(num_step_set2set))
    num_layer_set2set = int(round(num_layer_set2set))
    # batch_size = int(round(batch_size))
    learning_rate=learning_rate
    callback = dc.models.ValidationCallback(val, 500, metric)
    # dropout = dropout
    model = MPNNModel(
            n_tasks=1,
            mode='regression',
            batch_size=16,
            node_out_feats=node_out_feats,
            edge_hidden_feats=edge_hidden_feats,
            num_step_message_passing=num_step_message_passing,
            num_step_set2set=num_step_set2set,
            num_layer_set2set=num_layer_set2set,
            learning_rate=learning_rate,
            # configproto=config,
            **params
            )
    model.fit(train, callbacks=callback, nb_epoch=50)

    # MAE
    valid_scores_mae = model.evaluate(val, [mean_absolute_error], per_task_metrics=False)
    # R2
    valid_scores_r2 = model.evaluate(val, [pearson_r2_score], per_task_metrics=False)
    # RMSE
    valid_scores_rmse = model.evaluate(val, [rms_score], per_task_metrics=False)
    # -MAE
    neg_valid_scores_mae = -valid_scores_mae['mean-mean_absolute_error']

    neg_mae.append(neg_valid_scores_mae)
    mae.append(valid_scores_mae['mean-mean_absolute_error'])
    r2.append(valid_scores_r2['mean-pearson_r2_score'])
    rmse.append(valid_scores_rmse['mean-rms_score'])
    
    return neg_valid_scores_mae

In [ ]:


# config = tf.compat.v1.ConfigProto()
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True

tid = 'tid'+'12952'
dataset = csv_to_feat(tid)
train, val, test = split_data(dataset)
# Run the model 

neg_mae = []
mae = []
r2  = []
rmse = []
cov = matern32()
gp  = GaussianProcess(cov)
acq = Acquisition(mode='ExpectedImprovement')
tasks = ['pki']

gpgo = GPGO(gp, acq, hyper_model_mpnn, params_dict)

# Adjust the number of iterations, e.g., max_iter=100 and init_eval=10.
max_iter = 10
init_evals = 1
gpgo.run(max_iter=max_iter, init_evals=init_evals)


In [ ]:
# hyperparameter tuning
"""
from deepchem.models import GCNModel

def hyper_model(dense_layer_size, graph_conv_layer_size, batch_size, dropout, nb_epoch, **params):
     
    global neg_mae
    global mae
    global r2
    global rmse
    global config
    
    dense_layer_size = int(round(dense_layer_size))
    graph_conv_depth = 2 # two graph convolutional layers
    graph_conv_layer_size = int(round(graph_conv_layer_size))
    batch_size = int(round(batch_size))
    dropout = dropout
    model = GCNModel(
            n_tasks=1,
            graph_conv_layers=[graph_conv_layer_size]*graph_conv_depth,
            batch_size=batch_size,
            dropout=dropout,
            mode='regression',
            dense_layer_size=dense_layer_size,
            configproto=config,
            **params
            )
    model.fit(train)

    # MAE
    valid_scores_mae = model.evaluate(val, [mean_absolute_error], per_task_metrics=False)
    # R2
    valid_scores_r2 = model.evaluate(val, [pearson_r2_score], per_task_metrics=False)
    # RMSE
    valid_scores_rmse = model.evaluate(val, [rms_score], per_task_metrics=False)
    # -MAE
    neg_valid_scores_mae = -valid_scores_mae['mean-mean_absolute_error']

    neg_mae.append(neg_valid_scores_mae)
    mae.append(valid_scores_mae['mean-mean_absolute_error'])
    r2.append(valid_scores_r2['mean-pearson_r2_score'])
    rmse.append(valid_scores_rmse['mean-rms_score'])
    
    return neg_valid_scores_mae
"""